INSTALL DEPENDENCIES

In [8]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install evaluate
!pip install numpy as np
!pip install requests
!pip install bs4

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


LOAD BART-LARGE-CNN MODEL AND CNN_DAILYMAIL TEST DATASET FOR EVALUATION

In [9]:
from transformers import pipeline

bart_pipe = pipeline("summarization", model = "facebook/bart-large-cnn")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
from datasets import load_dataset
ds_test = load_dataset("abisee/cnn_dailymail","3.0.0", split = "test")

EVALUATE BART-LARGE-CNN MODEL USING BERTSCORE

In [11]:
no_eval_articles = 50

In [12]:
def chunked_text(text, chunk_size):
    chunks = []
    for i in range(0,len(text), chunk_size):
        chunk = text[i:i+chunk_size]
        chunks.append(chunk)
    return chunks

In [21]:
def summarize(bart_pipe, text, chunk_size, chunk_summary_size=128):
    chunks = chunked_text(text,chunk_size)

    summaries = []

    for chunk in chunks:
      size = min(chunk_summary_size, len(chunk)//2)

      summary = bart_pipe(chunk,max_length = size, min_length = 1, do_sample = False)[0]['summary_text']

      summaries.append(summary)

    return ' '.join(summaries)


In [22]:
bart_summaries = []
ref_summaries = []
CHUNK_SIZE = 1024
for i in range(no_eval_articles):
    article = ds_test[i]['article']
    summary = ds_test[i]['highlights']
    # SUMMARIZE
    bart_summary = summarize(bart_pipe,article,CHUNK_SIZE)

    bart_summaries.append((article, bart_summary))
    ref_summaries.append((article, summary))

Your max_length is set to 128, but your input_length is only 108. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)
Your max_length is set to 128, but your input_length is only 65. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)
Your max_length is set to 16, but your input_length is only 9. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)
Your max_length is set to 83, but your input_length is only 35. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)
Your ma

KeyboardInterrupt: 

In [ ]:
from evaluate import load

bert_score = load("bertscore")

results = bert_score.compute(predictions=bart_summaries, references=ref_summaries, model_type="facebook/bart-large-cnn")

f1s = results['f1']
precisions = results['precision']

In [ ]:
print(f"F1: {np.average(f1s,4)}")
print(f"Precisions: {np.average(precisions,4)}")

GET DATA FROM CNN WEBSITE AND SUMMARIZE IT

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_cnn_article(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")

        # For CNN articles
        if "cnn.com" in url:
            title = soup.find('h1').get_text()
            article_body = soup.find_all('p', class_="paragraph inline-placeholder vossi-paragraph")
            content = " ".join([p.get_text() for p in article_body])
            return title, content
        else:
            return None,None

    else:
        print(f"Failed to retrieve the article. Status code: {response.status_code}")
        return None, None

In [ ]:
title, content = scrape_cnn_article("https://edition.cnn.com/2024/10/20/politics/mcdonalds-donald-trump-pennsylvania/index.html")

data ={
    'title': title,
    'article': content,
}

In [ ]:
cnn_summary = summarize(bart_pipe,data['article'],CHUNK_SIZE)

In [ ]:
print(cnn_summary)